In [3]:
import os
import sys
import xml.etree.ElementTree as ET
import urllib.request as url_request
import urllib.parse as url_parse
import urllib.error as url_error
import pandas as pd

In [4]:
urlopen = url_request.urlopen
quote_plus = url_parse.quote_plus
urlencode = url_parse.urlencode
HTTPError = url_error.HTTPError

In [16]:
nan_char = '.'

In [5]:
root_url = 'https://api.stlouisfed.org/fred'

In [6]:
api_key='bf7e372a4731983197a88028a0d6c23c'

In [7]:
series_id = 'GDPC1'

In [8]:
url = "%s/series/observations?series_id=%s" % (root_url, series_id)

In [9]:
url

'https://api.stlouisfed.org/fred/series/observations?series_id=GDPC1'

In [10]:
url += '&api_key=' + api_key

In [11]:
url

'https://api.stlouisfed.org/fred/series/observations?series_id=GDPC1&api_key=bf7e372a4731983197a88028a0d6c23c'

In [13]:
try:
    response = urlopen(url)
    root = ET.fromstring(response.read())
except HTTPError as exc:
    root = ET.fromstring(exc.read())
    raise ValueError(root.get('message'))

print(root)

<Element 'observations' at 0x7f3b82a0acc8>


In [14]:
data = {}

In [20]:
def parse_datetime(date_str, format='%Y-%m-%d'):
    rv = pd.to_datetime(date_str, format=format)
    if hasattr(rv, 'to_pydatetime'):
        rv = rv.to_pydatetime()
    return rv

In [23]:
for child in root.getchildren():
    val = child.get('value')
    if val == nan_char:
        val = float('NaN')
    else:
        val = float(val)
    data[parse_datetime(child.get('date'))] = val

print(pd.Series(data))

1947-01-01     1934.471
1947-04-01     1932.281
1947-07-01     1930.315
1947-10-01     1960.705
1948-01-01     1989.535
1948-04-01     2021.851
1948-07-01     2033.155
1948-10-01     2035.329
1949-01-01     2007.522
1949-04-01     2000.788
1949-07-01     2022.798
1949-10-01     2004.653
1950-01-01     2084.559
1950-04-01     2147.592
1950-07-01     2230.362
1950-10-01     2273.435
1951-01-01     2304.542
1951-04-01     2344.517
1951-07-01     2392.808
1951-10-01     2398.105
1952-01-01     2423.457
1952-04-01     2428.516
1952-07-01     2446.081
1952-10-01     2526.359
1953-01-01     2573.372
1953-04-01     2593.456
1953-07-01     2578.922
1953-10-01     2539.786
1954-01-01     2528.043
1954-04-01     2530.689
                ...    
2010-04-01    14745.933
2010-07-01    14845.458
2010-10-01    14939.001
2011-01-01    14881.301
2011-04-01    14989.555
2011-07-01    15021.149
2011-10-01    15190.255
2012-01-01    15291.035
2012-04-01    15362.415
2012-07-01    15380.802
2012-10-01    15